In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# Cleaning Data

**Use the pd.read_csv() function to read the Online Retail.xlsx file into a dataframe called df**

In [2]:
df = pd.read_csv('OnlineRetail2.csv', encoding ='latin')

**Use the DataFrame.info() and DataFrame.head() methods to print information about both
dataframes, as well as the first few rows**

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,539993,22386,JUMBO BAG PINK POLKADOT,10,2011-01-04 10:00:00,1.95,13313.0,United Kingdom
1,539993,21499,BLUE POLKADOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom
2,539993,21498,RED RETROSPOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom
3,539993,22379,RECYCLING BAG RETROSPOT,5,2011-01-04 10:00:00,2.10,13313.0,United Kingdom
4,539993,20718,RED RETROSPOT SHOPPER BAG,10,2011-01-04 10:00:00,1.25,13313.0,United Kingdom


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203422 entries, 0 to 203421
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    203422 non-null  object 
 1   StockCode    203422 non-null  object 
 2   Description  202623 non-null  object 
 3   Quantity     203422 non-null  int64  
 4   InvoiceDate  203422 non-null  object 
 5   UnitPrice    203422 non-null  float64
 6   CustomerID   150039 non-null  float64
 7   Country      203422 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 12.4+ MB


**Check duplicated rows**

In [5]:
df.duplicated().sum()

1512

**Check the distribution of the Quantity attribute using the pd.describe() method**

In [6]:
df['Quantity'].describe()

count    203422.000000
mean          9.585684
std         240.921315
min      -74215.000000
25%           1.000000
50%           3.000000
75%          10.000000
max       74215.000000
Name: Quantity, dtype: float64

**What anomalies have you observed?**

The negative value appears in the minium of Quantity

- Drop duplicated rows 

- Delete records with the value of Quantity <=0 

- Drop records with null CustomerID

In [7]:
df = df.drop_duplicates()

df = df.drop_duplicates(subset=['InvoiceNo', 'StockCode', 'Quantity'])
df = df[df['CustomerID'].notnull()]
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

df = df[df['Quantity'] > 0]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145000 entries, 0 to 203421
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    145000 non-null  object        
 1   StockCode    145000 non-null  object        
 2   Description  145000 non-null  object        
 3   Quantity     145000 non-null  int64         
 4   InvoiceDate  145000 non-null  datetime64[ns]
 5   UnitPrice    145000 non-null  float64       
 6   CustomerID   145000 non-null  float64       
 7   Country      145000 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 10.0+ MB


# RFM Analysis

**Calculate Recency, Frequency, and Monetary values for each customer. Store the result in a dataframe named the customer_df**

In [9]:
customer_df = pd.DataFrame()
type(customer_df)

pandas.core.frame.DataFrame

In [10]:
customer_df['CustomerID'] = df['CustomerID'].unique()

**Create a new column named ‘TotalRevenue’ as the product of two columns Quantity and UnitPrice**

In [11]:
df['TotalRevenue'] = df['Quantity'] * df['UnitPrice']
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalRevenue
0,539993,22386,JUMBO BAG PINK POLKADOT,10,2011-01-04 10:00:00,1.95,13313.0,United Kingdom,19.50
1,539993,21499,BLUE POLKADOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom,10.50
2,539993,21498,RED RETROSPOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom,10.50
3,539993,22379,RECYCLING BAG RETROSPOT,5,2011-01-04 10:00:00,2.10,13313.0,United Kingdom,10.50
4,539993,20718,RED RETROSPOT SHOPPER BAG,10,2011-01-04 10:00:00,1.25,13313.0,United Kingdom,12.50
...,...,...,...,...,...,...,...,...,...
203417,558637,22032,BOTANICAL LILY GREETING CARD,12,2011-06-30 20:08:00,0.42,17891.0,United Kingdom,5.04
203418,558637,22028,PENNY FARTHING BIRTHDAY CARD,12,2011-06-30 20:08:00,0.42,17891.0,United Kingdom,5.04
203419,558637,22033,BOTANICAL ROSE GREETING CARD,12,2011-06-30 20:08:00,0.42,17891.0,United Kingdom,5.04
203420,558637,22029,SPACEBOY BIRTHDAY CARD,12,2011-06-30 20:08:00,0.42,17891.0,United Kingdom,5.04


**Create a new dataframe named frequency_df. This dataframe has only two columns:**

    • The ‘CustomerID’ column stores the unique ID of each customer
    • The ‘Frequency’ column stores the frequency value of each customer

In [12]:
frequency_col = df.groupby('CustomerID')['InvoiceNo'].nunique()
frequency_col

CustomerID
12346.0    1
12347.0    3
12348.0    2
12350.0    1
12352.0    5
          ..
18273.0    1
18280.0    1
18281.0    1
18283.0    7
18287.0    1
Name: InvoiceNo, Length: 2724, dtype: int64

In [13]:
frequency_df = pd.DataFrame({'CustomerID':frequency_col.index, 'Frequency':frequency_col.values})
frequency_df

,CustomerID,Frequency
0,12346.0,1
1,12347.0,3
2,12348.0,2
3,12350.0,1
4,12352.0,5
...,...,...
2719,18273.0,1
2720,18280.0,1
2721,18281.0,1
2722,18283.0,7


**Create a new dataframe named monetary_df. This dataframe has only two columns:**
    
    • The ‘ CustomerID’ column stores the unique ID of each customer
    • The ‘Monetary’ column stores the total revenue of each customer


In [14]:
monetary_col = df.groupby('CustomerID')['TotalRevenue'].sum()
monetary_col

CustomerID
12346.0    77183.60
12347.0     1494.16
12348.0      594.44
12350.0      334.40
12352.0     1098.01
             ...   
18273.0       51.00
18280.0      180.60
18281.0       80.82
18283.0      812.04
18287.0      765.28
Name: TotalRevenue, Length: 2724, dtype: float64

In [15]:
monetary_df = pd.DataFrame({'CustomerID':monetary_col.index,'Monetary':monetary_col.values})
monetary_df

,CustomerID,Monetary
0,12346.0,77183.60
1,12347.0,1494.16
2,12348.0,594.44
3,12350.0,334.40
4,12352.0,1098.01
...,...,...
2719,18273.0,51.00
2720,18280.0,180.60
2721,18281.0,80.82
2722,18283.0,812.04


**Create a new dataframe named recency_df. This dataframe has only two columns:**

    • The ‘CustomerID’ column stores the unique ID of each customer
    • The ‘Recency’ column stores the recency value of each customer

In [16]:
recency_col =(df['InvoiceDate'].max() - df.groupby('CustomerID')['InvoiceDate'].max()).dt.days
recency_col

CustomerID
12346.0    163
12347.0     21
12348.0     86
12350.0    148
12352.0    100
          ... 
18273.0     95
18280.0    115
18281.0     18
18283.0      7
18287.0     39
Name: InvoiceDate, Length: 2724, dtype: int64

In [17]:
recency_df = pd.DataFrame({'CustomerID':recency_col.index, 'Recency':recency_col.values})
recency_df

,CustomerID,Recency
0,12346.0,163
1,12347.0,21
2,12348.0,86
3,12350.0,148
4,12352.0,100
...,...,...
2719,18273.0,95
2720,18280.0,115
2721,18281.0,18
2722,18283.0,7


In [18]:
customer_df = pd.merge(customer_df, frequency_df, on='CustomerID')
customer_df = pd.merge(customer_df, monetary_df, on='CustomerID')
customer_df = pd.merge(customer_df, recency_df, on='CustomerID')

In [19]:
customer_df

,CustomerID,Frequency,Monetary,Recency
0,13313.0,2,609.74,53
1,18097.0,2,1274.04,43
2,16656.0,5,3128.72,30
3,16875.0,3,805.09,134
4,13094.0,7,869.40,29
...,...,...,...,...
2719,15578.0,1,127.10,0
2720,15584.0,1,150.40,0
2721,13745.0,1,300.04,0
2722,16498.0,1,91.22,0


**Add segment bin values to RFM table using quartile. For example, If the recency value
belongs to the first quartile, the recency value will be replaced by 1. If it belongs to the
second quartile, that value will be replaced by 2**

• Hint: using pd.qcut, create new columns named r_quantile, f_quantile, and
m_quantile in the dataframe customer_df

• In the customer_df, Create a new column named RFM_Score(1 point). The formula
for RFM_Score is as follows
RFM_Score = r_quantile + f_quantile + m_quantile

In [20]:
# Create r_quantile column
customer_df['r_quantile'] = pd.qcut(customer_df.Recency,
                                    q=4,
                                    labels=[1,2,3,4])

In [21]:
# Create f_quantile column
customer_df['f_quantile'] = pd.qcut(customer_df.Frequency,
                                    q=4,
                                    labels=[1,2,3],
                                    duplicates = 'drop')

In [22]:
# Create m_quantile column
customer_df['m_quantile'] = pd.qcut(customer_df.Monetary,
                                    q=4,
                                    labels=[1,2,3,4])

In [23]:
customer_df

,CustomerID,Frequency,Monetary,Recency,r_quantile,f_quantile,m_quantile
0,13313.0,2,609.74,53,3,1,3
1,18097.0,2,1274.04,43,2,1,4
2,16656.0,5,3128.72,30,2,3,4
3,16875.0,3,805.09,134,4,2,3
4,13094.0,7,869.40,29,2,3,3
...,...,...,...,...,...,...,...
2719,15578.0,1,127.10,0,1,1,1
2720,15584.0,1,150.40,0,1,1,1
2721,13745.0,1,300.04,0,1,1,2
2722,16498.0,1,91.22,0,1,1,1


**In the customer_df, Create a new column named RFM_Score(1 point). The formula
for RFM_Score is as follows
RFM_Score = r_quantile + f_quantile + m_quantile**

In [24]:
customer_df['RFM_Score'] = customer_df['r_quantile'].astype(int) + customer_df['f_quantile'].astype(int) + customer_df['m_quantile'].astype(int)

In [25]:
customer_df

,CustomerID,Frequency,Monetary,Recency,r_quantile,f_quantile,m_quantile,RFM_Score
0,13313.0,2,609.74,53,3,1,3,7
1,18097.0,2,1274.04,43,2,1,4,7
2,16656.0,5,3128.72,30,2,3,4,9
3,16875.0,3,805.09,134,4,2,3,9
4,13094.0,7,869.40,29,2,3,3,8
...,...,...,...,...,...,...,...,...
2719,15578.0,1,127.10,0,1,1,1,3
2720,15584.0,1,150.40,0,1,1,1,3
2721,13745.0,1,300.04,0,1,1,2,4
2722,16498.0,1,91.22,0,1,1,1,3


**Based on RFM_Score, customers are divided into 3 segments: low-value, mid-value, and high-value so that it satisfies the following rules:**

    - The number of customers in the high-value segment does not exceed 20% of the total number of customers.
    
    - The number of customers in the mid-value segment is no less than 30% of the total number of customers

In [26]:
customer_df['Customer_Segments'] = pd.qcut(customer_df.RFM_Score,
                            q=[0,0.3,0.8,1.0],
                            labels =['low-value','mid-value','high-value'])

In [27]:
customer_df

,CustomerID,Frequency,Monetary,Recency,r_quantile,f_quantile,m_quantile,RFM_Score,Customer_Segments
0,13313.0,2,609.74,53,3,1,3,7,mid-value
1,18097.0,2,1274.04,43,2,1,4,7,mid-value
2,16656.0,5,3128.72,30,2,3,4,9,high-value
3,16875.0,3,805.09,134,4,2,3,9,high-value
4,13094.0,7,869.40,29,2,3,3,8,mid-value
...,...,...,...,...,...,...,...,...,...
2719,15578.0,1,127.10,0,1,1,1,3,low-value
2720,15584.0,1,150.40,0,1,1,1,3,low-value
2721,13745.0,1,300.04,0,1,1,2,4,low-value
2722,16498.0,1,91.22,0,1,1,1,3,low-value
